In [79]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pandas as pd

In [80]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [81]:
cap = cv2.VideoCapture(0)

ls = []
with mp_hands.Hands(
    model_complexity = 0,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as hands:
    
    while cap.isOpened():

        time_start = time.time()

        retval, frame = cap.read()

        if not retval:
            print("Camera Error; Exiting")
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                
                ls.append([[lmk.x, lmk.y] for lmk in hand_landmarks.landmark])

                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        fps = 1 / (time.time() - time_start)

        frame = cv2.putText(frame, f"FPS: {round(fps)}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cv2.imshow("Hand Landmarks", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [82]:
# normalize coordinates to be relative centered at landmark 0
ls = np.array(ls)

for l in ls:
    for i in range(21):
        l[i][0] = l[i][0] - l[0][0]
        l[i][1] = l[i][1] - l[0][1]

ls

array([[[0.        , 0.        ],
        [0.41142672, 0.7185635 ],
        [0.43523899, 0.69026423],
        ...,
        [0.32168722, 0.60430151],
        [0.32055601, 0.57931131],
        [0.32561558, 0.55994016]],

       [[0.        , 0.        ],
        [0.40970194, 0.72171599],
        [0.43470708, 0.6897974 ],
        ...,
        [0.32395852, 0.60363406],
        [0.32129359, 0.57771444],
        [0.32575437, 0.55481452]],

       [[0.        , 0.        ],
        [0.40711507, 0.72399193],
        [0.43260297, 0.69002175],
        ...,
        [0.32487231, 0.6053409 ],
        [0.32100973, 0.57966065],
        [0.32384011, 0.55561447]],

       ...,

       [[0.        , 0.        ],
        [0.4458611 , 0.6374653 ],
        [0.48108494, 0.59852004],
        ...,
        [0.36219814, 0.48363525],
        [0.35435003, 0.45422176],
        [0.35027289, 0.42595607]],

       [[0.        , 0.        ],
        [0.44369343, 0.64151651],
        [0.47889656, 0.60449022],
        .

In [83]:
reshaped_ls = ls.copy()
reshaped_ls = reshaped_ls.reshape(-1, 42)
reshaped_ls.shape

(664, 42)

In [84]:
df_columns = []
for i in range(21):
    df_columns.append(f"Landmark {i} x")
    df_columns.append(f"Landmark {i} y")

In [85]:
# create dataframe where rows are frames and columns are coordinates of all 21 landmarks (x, y have distinct columns)
df = pd.DataFrame(reshaped_ls, columns=df_columns)
df.drop(["Landmark 0 x", "Landmark 0 y"], axis=1, inplace=True) # can delete landmark 0 since its all just 0 anyway
df.shape

(664, 40)

In [86]:
# add label for handpose (DO NOT USE EMOJIS LOL)
df["Label"] = "👌"
df

,Landmark 1 x,Landmark 1 y,Landmark 2 x,Landmark 2 y,Landmark 3 x,Landmark 3 y,Landmark 4 x,Landmark 4 y,Landmark 5 x,Landmark 5 y,...,Landmark 16 y,Landmark 17 x,Landmark 17 y,Landmark 18 x,Landmark 18 y,Landmark 19 x,Landmark 19 y,Landmark 20 x,Landmark 20 y,Label
0,0.411427,0.718563,0.435239,0.690264,0.451207,0.665588,0.460732,0.641190,0.397451,0.609153,...,0.506451,0.339447,0.635135,0.321687,0.604302,0.320556,0.579311,0.325616,0.559940,👌
1,0.409702,0.721716,0.434707,0.689797,0.452413,0.664166,0.457258,0.639000,0.397381,0.609898,...,0.507120,0.342739,0.639006,0.323959,0.603634,0.321294,0.577714,0.325754,0.554815,👌
2,0.407115,0.723992,0.432603,0.690022,0.452056,0.664563,0.459533,0.642614,0.398356,0.609807,...,0.503982,0.344339,0.640477,0.324872,0.605341,0.321010,0.579661,0.323840,0.555614,👌
3,0.406369,0.724119,0.431909,0.689245,0.451636,0.664142,0.458694,0.641510,0.397826,0.611085,...,0.504440,0.343721,0.638862,0.324308,0.605070,0.321006,0.579423,0.324600,0.555556,👌
4,0.407705,0.719752,0.434439,0.685973,0.455001,0.661350,0.461353,0.640044,0.399168,0.607969,...,0.496347,0.345749,0.636528,0.325553,0.601953,0.320859,0.575897,0.322466,0.550252,👌
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,0.445534,0.634102,0.480494,0.596417,0.497962,0.561720,0.490642,0.536983,0.461491,0.515201,...,0.374262,0.380604,0.522982,0.363917,0.481397,0.357002,0.452227,0.353727,0.423873,👌
660,0.445572,0.637749,0.480321,0.599611,0.496844,0.565749,0.487908,0.544006,0.460924,0.516484,...,0.376993,0.379810,0.526149,0.362807,0.484110,0.355155,0.455340,0.351173,0.427827,👌
661,0.445861,0.637465,0.481085,0.598520,0.497399,0.563595,0.487667,0.540827,0.460589,0.517133,...,0.376158,0.379290,0.525908,0.362198,0.483635,0.354350,0.454222,0.350273,0.425956,👌
662,0.443693,0.641517,0.478897,0.604490,0.497372,0.570915,0.490183,0.548214,0.458703,0.520641,...,0.379582,0.378575,0.529554,0.361231,0.488517,0.352885,0.459342,0.348170,0.431157,👌


In [87]:
try:
    whole = pd.concat([whole, df], axis=0)
except:
    whole = df

whole.shape

(1240, 41)

In [88]:
whole.to_csv("signs.csv")